In [2]:
import pandas as pd
import numpy as np
import statistics

In [3]:
import requests
import json

In [4]:
def callApi(method, stock,isEarnings = False ):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last = "/10"
    period = "period=annual"
    if isEarnings:
        url = host + ticker + api_method + last + query + period + add + key 
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

In [5]:
def peRatio(quote):
    if "peRatio" in quote.keys() and quote["peRatio"] is not None:
        return float(quote["peRatio"])
    else:
        0.0

In [6]:
def priceToBook(stats):
    if "priceToBook" in stats.keys() and stats["priceToBook"] is not None:
        return float(stats["priceToBook"])
    else:
        return 0.0

In [7]:
def debtToEquity(stats):
    if "debtToEquity" in stats.keys() and stats["debtToEquity"] is not None:
        return float(stats["debtToEquity"])
    else:
        return 0.0
    

In [8]:
def pegRatio(stats):
    if not("pegRatio" in stats.keys()):
        return 0.0
    if stats["pegRatio"] is not None:
        return float(stats["pegRatio"])
    else:
        return 0.0

In [9]:
def sharesoutstanding(stats):
    if not("sharesOutstanding" in stats.keys()):
        return 0.0
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    else:
        return 0.0

In [10]:
# Current Ratio= Current Assets / Current Liabilities
def currentRatio(balance_sheet):
    if not("balancesheet" in balance_sheet.keys()) or len(balance_sheet["balancesheet"]) == 0:
        return 0.0
    latest_balance_sheet = balance_sheet["balancesheet"][0]
    if not("totalAssets" in latest_balance_sheet.keys()) and not("totalLiabilities" in latest_balance_sheet.keys()):
        return 0.0
    return float(latest_balance_sheet["totalAssets"]/ latest_balance_sheet["totalLiabilities"])
    

In [11]:
def checkErningProgression(earnings):
    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return False
    earningsSet = earnings["earnings"][:-1]
    yearAgoChangePercent = np.array([])
    for earning in earningsSet:
        if not("yearAgoChangePercent" in earning.keys()):
            return False
        yearAgoChangePercent = np.append(yearAgoChangePercent, earning["yearAgoChangePercent"])
    isDecending = (yearAgoChangePercent == np.sort(yearAgoChangePercent)[::-1]).all()
    return isDecending

In [12]:
tickers = ['HTZ']
dataSet = [] 
for ticker in tickers:
    data = {}
    points = 0
    data["Ticker"] = ticker
    # Major data set 
    stats = callApi("advanced-stats", ticker)
    earnings = callApi("earnings", ticker, True)
    earning = callApi("earnings", ticker)
    #earnings["earnings"].insert(0,earning["earnings"][0])
    price = callApi("price", ticker)
    
    data["Price"] = price
    # PE Ratio < 15
    pe_ratio = peRatio(quote)
    if pe_ratio != 0: 
        data["PE Ratio < 15"] = pe_ratio
    else: 
        data["PE Ratio < 15"] = "NA"

    # Price To Book Ratio  
    price_to_book = priceToBook(stats)
    if price_to_book != 0: 
        data["Price To Book < 1.5"] = price_to_book
    else: 
        data["Price To Book < 1.5"] = "NA"
    
    if pe_ratio * price_to_book < 22.5 and pe_ratio * price_to_book > 0: 
        points += 1
    
    # PEG Ratio 
    peg_ratio = pegRatio(stats)
    if peg_ratio != 0: 
        data["PEG Ratio(1 and 0)"] = peg_ratio
    else: 
        data["PEG Ratio(1 and 0)"] = "NA"
    if peg_ratio < 1.1 and peg_ratio > 0:
        points += 1
    
    # Debt To Equity Ratio 
    # Warent buffet say < 0.5
    debt_Equity = debtToEquity(stats)
    if debt_Equity != 0: 
        data["Debt to Equity (0 and 1.5)"] = debt_Equity
    else: 
        data["Debt to Equity (0 and 1.5)"] = "NA"
    if debt_Equity < 1.6 and debt_Equity > 0:
        points += 1
        
    # Current Ratio 
    current_ratio = currentRatio(balance_sheet)
    if current_ratio != 0: 
        data["Current Ratio > 1.5"] = current_ratio
    else: 
        data["Current Ratio > 1.5"] = "NA"
        
    if current_ratio > 1.4:
        points += 1
    
    # Earning Progression Last 4 year 
    isProgressing = checkErningProgression(earnings)
    data["Earning Progression"] =  isProgressing
    if isProgressing:
        points += 1
    
    data["Points(5)"] = points

    dataSet.append(data.copy())
pd.DataFrame(dataSet)    

,Ticker,Price,PE Ratio < 15,Price To Book < 1.5,PEG Ratio(1 and 0),Debt to Equity (0 and 1.5),Current Ratio > 1.5,Earning Progression,Points(5)
0,HTZ,1.36,-0.17,0.120091,NA,10.7,1.029987,True,1
